In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
import datetime

# Data Loading Block

In [3]:
df_pnr_book=pd.read_csv("/content/drive/MyDrive/TEAM-69/Final Dataset/PNRB-ZZ-20231208_062017.csv")
df_pnr_pass=pd.read_csv('/content/drive/MyDrive/TEAM-69/Final Dataset/PNRP-ZZ-20231208_111136.csv')
df_schedule = pd.read_csv('/content/drive/MyDrive/TEAM-69/Final Dataset/SCH-ZZ-20231208_035117.csv')
df_inventory = pd.read_csv('/content/drive/MyDrive/TEAM-69/Final Dataset/INV-ZZ-20231208_041852.csv')

# Data Preprocessing Block

In [4]:
date_columns = ['DEP_DTML', 'ARR_DTML', 'DEP_DTMZ', 'ARR_DTMZ']
for column in date_columns:
    df_pnr_book[column] = pd.to_datetime(df_pnr_book[column], errors='coerce').dt.strftime('%m/%d/%Y %H:%M')

df_pnr_book['DEP_DT'] = pd.to_datetime(df_pnr_book['DEP_DT'], errors='coerce').dt.strftime('%m/%d/%Y')


#For Inventory

df_inventory['DepartureDate'] = pd.to_datetime(df_inventory['DepartureDate'], errors='coerce').dt.strftime('%m/%d/%Y')
date_columns = ['DepartureDateTime', 'ArrivalDateTime']
for column in date_columns:
    df_inventory[column] = pd.to_datetime(df_inventory[column], errors='coerce').dt.strftime('%Y/%m/%d %H:%M:%S')

date_columns = ['DepartureDateTime', 'ArrivalDateTime']
df_inventory[date_columns] = df_inventory[date_columns].apply(pd.to_datetime, format='%Y-%m-%d %H:%M:%S')

In [5]:
df_pnr_book_copy=df_pnr_book.copy()
df_pnr_book_copy1=df_pnr_book.copy()
df_pnr_pass_copy=df_pnr_pass.copy()
df_pnr_pass_copy1=df_pnr_pass.copy()
df_schedule_copy=df_schedule.copy()
df_schedule_copy1=df_schedule.copy()
df_inventory_copy=df_inventory.copy()
df_inventory_copy1=df_inventory.copy()

# Input Block

Example Input:
cancelled_FN = 2019,
cancelled_TN = 'VT-7102',
departure_date = '04/21/2024'

In [6]:
cancelled_FN = 2019
cancelled_TN = 'VT-7102'
departure_date = '04/21/2024'

In [ ]:
cancelled_FN = int(input("Enter the flight number: "))

# Taking user input for tail_number as a string
cancelled_TN= input("Enter the tail number: ")

# Taking user input for date as a string
departure_date= input("Enter the date (MM/DD/YYYY): ")

Enter the flight number: 2019
Enter the tail number: VT-7102
Enter the date (MM/DD/YYYY): 04/21/2024


# The 1-1 Reallocation Function Block(Prioritizing the classes of Passengers)

Our goal is to prioritize the reaccommodation of passengers according to their classes, even if it requires making some adjustments to the accommodation based on the priority of flights.

In [7]:
def datframe_allocation(df_pnr_pass, df_pnr_book, df_schedule,df_inventory, cancelled_FN, cancelled_TN, departure_date):
    def get_available_flights_1_1(schedule_df, inventory_df, cancelled_FN, cancelled_TN, departure_date):
        cancelled_schedule = df_schedule[(df_schedule['AircraftTailNumber'] == cancelled_TN) & (df_schedule['FlightNumber'] == cancelled_FN)]
        print(cancelled_schedule['ScheduleID'].iloc[0])

        cancelled_flight = df_inventory[(df_inventory['ScheduleId'] == cancelled_schedule['ScheduleID'].iloc[0]) & (df_inventory['DepartureDate'] == departure_date)]
        cancelled_flight_DepartureDateTime = cancelled_flight['DepartureDateTime'].iloc[0]
        cancelled_flight_ArrivalDateTime = cancelled_flight['ArrivalDateTime'].iloc[0]

        matching_flights = df_inventory[
        (df_inventory['DepartureAirport'] == cancelled_flight['DepartureAirport'].iloc[0]) &
        (df_inventory['ArrivalAirport'] == cancelled_flight['ArrivalAirport'].iloc[0]) &
        (df_inventory['DepartureDateTime'] > cancelled_flight_DepartureDateTime)
        ]

        columns_to_keep=['InventoryId', 'ScheduleId', 'CarrierCode', 'Dep_Key', 'FlightNumber', 'AircraftType', 'DepartureDate', 'DepartureDateTime', 'ArrivalDateTime', 'DepartureAirport', 'ArrivalAirport', 'TotalCapacity', 'TotalInventory','BookedInventory', 'Oversold', 'AvailableInventory', 'FirstClass', 'BusinessClass', 'PremiumEconomyClass', 'EconomyClass', 'FC_TotalInventory', 'FC_BookedInventory', 'FC_Oversold', 'FC_AvailableInventory', 'BC_TotalInventory', 'BC_BookedInventory', 'BC_Oversold', 'BC_AvailableInventory', 'PC_TotalInventory', 'PC_BookedInventory', 'PC_Oversold', 'PC_AvailableInventory', 'EC_TotalInventory', 'EC_BookedInventory', 'EC_Oversold', 'EC_AvailableInventory', 'FC_CD', 'BC_CD', 'PC_CD', 'EC_CD']
        matching_flights = matching_flights[columns_to_keep]

        matching_flights['Arrival_TimeDifference'] = (matching_flights['ArrivalDateTime'] - cancelled_flight_ArrivalDateTime).dt.total_seconds()/3600
        matching_flights = matching_flights.sort_values(by='Arrival_TimeDifference')
        print(matching_flights.shape)
        return matching_flights

    def process_pnr_data(df_pnr_pass, df_pnr_book, df_schedule, flight_number, tail_number, date):
        dept_time = df_schedule[(df_schedule['FlightNumber'] == flight_number) & (df_schedule['AircraftTailNumber'] == tail_number)]['DepartureTime'].iloc[0]
        date_time = date + ' ' + dept_time
        print(date_time)

        def cal_affected_persons(flt_number, date_time):
            dict={}
            affect_persons = pd.DataFrame(columns=df_pnr_book.columns)

            for i in range(len(df_pnr_book)):
                if df_pnr_book.loc[i, "RECLOC"] in dict.keys():
                    dict[df_pnr_book.loc[i, "RECLOC"]] = max(dict[df_pnr_book.loc[i, "RECLOC"]], df_pnr_book.loc[i, "SEG_SEQ"])
                else:
                    dict[df_pnr_book.loc[i, "RECLOC"]] = df_pnr_book.loc[i, "SEG_SEQ"]

                if df_pnr_book.loc[i, "FLT_NUM"] == flt_number and df_pnr_book.loc[i, "DEP_DTML"] == date_time:
                    affect_persons = affect_persons.append(df_pnr_book.loc[i])
            li = []
            for i in range(len(affect_persons)):
                if(dict[affect_persons.iloc[i]["RECLOC"]]):
                     li.append(dict[affect_persons.iloc[i]["RECLOC"]])
            affect_persons["MAX_SEG"] = li
            return affect_persons

        def calc_ssr(flight_number, date_time):
            affect_person = cal_affected_persons(flight_number, date_time)
            li = []
            for i in range(len(affect_person)):
                li.append(0)
            affect_person["count_ssr"] = li
            for i in range(len(affect_person)):
                for j in range(len(df_pnr_pass)):
                    if df_pnr_pass.loc[j, "RECLOC"] == affect_person.iloc[i]["RECLOC"]:
                        if pd.notna(df_pnr_pass["SSR_CODE_CD1"][j]) or pd.notna(df_pnr_pass["SPECIAL_NAME_CD2"][j]):
                            affect_person.iloc[i]["count_ssr"] += 1
            return affect_person

        def score_eval(affect_PNR):
            score = 0
            score += 200 * affect_PNR.loc["count_ssr"] + 50 * affect_PNR.loc["PAX_CNT"]
            score += 100 * (affect_PNR.loc["MAX_SEG"] - affect_PNR.loc["SEG_SEQ"])
            score_dict = {'A': 'J', 'D': 'J', 'J': 'J', 'B': 'F', 'F': 'F', 'S': 'Y', 'V': 'Y', 'W': 'Y', 'Z': 'Y', 'O': 'Y',
                      'S': 'Y', 'T': 'Y', 'U': 'Y', 'M': 'Y', 'N': 'Y', 'Y': 'Y'}
            if affect_PNR.loc["COS_CD"] not in score_dict.keys():
                score += 500
            elif affect_PNR.loc["COS_CD"] == 'J' or score_dict[affect_PNR.loc["COS_CD"]] == 'J':
                score += 2000
            elif affect_PNR.loc["COS_CD"] == "F" or score_dict[affect_PNR.loc["COS_CD"]] == "F":
                score += 1700
            elif affect_PNR.loc["COS_CD"] == "Y" or score_dict[affect_PNR.loc["COS_CD"]] == "Y":
                score += 1500
            elif affect_PNR.loc["COS_CD"] in ["A", "C", "K"]:
                score += 800
            return score

        def score(flight, data_time):
            affect_PNR = calc_ssr(flight, data_time)
            pnr_ranking = pd.DataFrame(columns=affect_PNR.columns.tolist() + ['score'])
            for i in range(len(affect_PNR)):
                score_val = score_eval(affect_PNR.iloc[i])
                affect_row = list(affect_PNR.iloc[i])
                affect_row.append(score_val)
                pnr_ranking = pnr_ranking.append(pd.Series(affect_row, index=pnr_ranking.columns), ignore_index=True)

            sorted_pnr_ranking = pnr_ranking.sort_values(by='score', ascending=False)
            return sorted_pnr_ranking
        return score(flight_number, date_time)

    def get_ITN_DEST(df):
        df[['SEG_SEQ', 'SEG_TOTAL']] = df[['SEG_SEQ', 'SEG_TOTAL']].apply(pd.to_numeric, errors='coerce')
        max_dest_per_recloc = df.loc[df['SEG_SEQ'] == df['SEG_TOTAL']].groupby('RECLOC')['DEST_CD'].first().reset_index()
        df = pd.merge(df, max_dest_per_recloc, on='RECLOC', how='left', suffixes=('', '_ITN_DEST'))
        return df
    def get_cluster_rank(affect_sorted_PNR):
        cluster_means = affect_sorted_PNR.groupby('DEST_CD_ITN_DEST')['score'].mean()
        rank_mapping = {cluster: rank for rank, (cluster, _) in enumerate(cluster_means.sort_values(ascending=False).iteritems(), start=1)}
        affect_sorted_PNR['CLUSTER_RANK'] = affect_sorted_PNR['DEST_CD_ITN_DEST'].map(rank_mapping)
        return affect_sorted_PNR

    def get_pnr_affected(df_pnr_pass, df_pnr_book, df_schedule, flight_number, tail_number, date):
        df_pnr_book = get_ITN_DEST(df_pnr_book)
        affect_sorted_PNR = process_pnr_data(df_pnr_pass, df_pnr_book, df_schedule, flight_number, tail_number, date)
        final_affected_PNR = get_cluster_rank(affect_sorted_PNR)
        return final_affected_PNR
    def knapsack1(left_seats, n, li, PNR_df, memo):
        if n == 0 or left_seats == 0:
            return left_seats, li

    # Check if the result is already memoized
        if (left_seats, n) in memo:
             return memo[(left_seats, n)]

        if left_seats >= PNR_df.iloc[n-1]["PAX_CNT"]:
        # Recursively compute the result and memoize it
            result1 = knapsack1(left_seats, n-1, li, PNR_df, memo)
            result2 = knapsack1(left_seats - PNR_df.iloc[n-1]["PAX_CNT"], n-1, li + [PNR_df.iloc[n-1]["RECLOC"]], PNR_df, memo)
            result = min(result1, result2, key=lambda x: x[0])
        else:
            result = knapsack1(left_seats, n-1, li, PNR_df, memo)

    # Memoize the result
        memo[(left_seats, n)] = result

        return result

    # Example usag  # Replace with your actual DataFrame

    def delete_entry(to_delete, dataframe):
        # print(dataframe)
        for i in to_delete:
            dataframe.drop(dataframe[dataframe["RECLOC"]==i].index, inplace=True, axis=0)
        return dataframe

    def all_flights_knapsack_for_same_class1(df_PNR, flights, cls):
        prev_cls=cls
        allocated_dict = {}
        for i in range(len(flights)):
            seats_left, allocated_PNRs = knapsack1(flights.iloc[i][cls+"_AvailableInventory"], len(df_PNR), [], df_PNR, {})

        # Update the values in the flights DataFrame
            temp_column = cls+"_AvailableInventory"
            temp = flights.iloc[i][temp_column]
            column_index = flights.columns.get_loc(temp_column)
            flights.iat[i, column_index] = seats_left
            flights.iat[i, flights.columns.get_loc("AvailableInventory")] -= temp - seats_left
            df_PNR = delete_entry(allocated_PNRs, df_PNR)
            allocated_dict[flights.iloc[i]["InventoryId"]] = [(pnr, cls,prev_cls) for pnr in allocated_PNRs]

        return allocated_dict

    def merge_dict(dict1,dict2):
        list1=dict1.keys()
        for i in list1:
            dict1[i]=dict1[i]+dict2[i]
        return dict1

    def convert_dict_to_dataframe(allocated_dict):
        data = []
        for inventory_id, allocations in allocated_dict.items():
            for allocation in allocations:
                pnr, class_name,prev_clssname = allocation
                data.append({'PNR': pnr, 'InventoryId': inventory_id, 'New_ClassName': class_name,'Prev_ClassName': prev_clssname})

        df = pd.DataFrame(data)
        return df

    def complete_allocation(df_FC_copy,df_BC_copy,df_PE_copy,df_EC_copy,flights):
        # FC_allocation={}
        # BC_allocation={}
        # PE_allocation={}
        # EC_allocation={}
        FC_allocation=all_flights_knapsack_for_same_class1(df_FC_copy, flights, "FC")
        BC_allocation=all_flights_knapsack_for_same_class1(df_BC_copy, flights, "BC")
        PE_allocation=all_flights_knapsack_for_same_class1(df_PE_copy, flights, "PC")
        EC_allocation=all_flights_knapsack_for_same_class1(df_EC_copy, flights, "EC")
        # print(FC_allocation)
        # print(BC_allocation)
        print(PE_allocation)
        # print(EC_allocation)


        if not df_FC_copy.empty:
            PE_allocation.update(all_flights_knapsack_for_same_class1(df_FC_copy, flights, "PC"))
            EC_allocation.update(all_flights_knapsack_for_same_class1(df_FC_copy, flights, "EC"))
        if not df_BC_copy.COS_CD.empty:
            EC_allocation.update(all_flights_knapsack_for_same_class1(df_BC_copy, flights, "EC"))
        if not df_PE_copy.empty:
            FC_allocation.update(all_flights_knapsack_for_same_class1(df_PE_copy, flights, "FC"))
        if not df_EC_copy.empty:
            FC_allocation.update(all_flights_knapsack_for_same_class1(df_EC_copy, flights, "FC"))
            BC_allocation.update(all_flights_knapsack_for_same_class1(df_EC_copy, flights, "BC"))
            PE_allocation.update(all_flights_knapsack_for_same_class1(df_EC_copy, flights, "PC"))

        # return  FC_allocation,BC_allocation,PE_allocation,EC_allocation
        df=convert_dict_to_dataframe(FC_allocation)
        df=pd.concat([df, convert_dict_to_dataframe(BC_allocation)], ignore_index=True)
        df=pd.concat([df, convert_dict_to_dataframe(PE_allocation)], ignore_index=True)
        df=pd.concat([df, convert_dict_to_dataframe(EC_allocation)], ignore_index=True)
        PNR_NOT_Allocated=df_FC_copy['RECLOC'].tolist()
        PNR_NOT_Allocated=PNR_NOT_Allocated+df_BC_copy['RECLOC'].tolist()+df_PE_copy['RECLOC'].tolist()+df_EC_copy['RECLOC'].tolist()
        new_df=pd.DataFrame({'PNR':PNR_NOT_Allocated})
        print(df)
        result_df = pd.merge(df, new_df, on='PNR', how='left')
        # result_df[['InventoryID','New_ClassName','Prev_ClassName']] = df[['InventoryID','New_ClassName','Prev_ClassName']]
        merged_df = pd.merge(df, flights, on='InventoryId', how='inner')
        result_df= new_df.set_index('PNR').combine_first(merged_df.set_index('PNR')).reset_index()
        return result_df

    def classification(affect_sorted_PNR):
        # Assuming 'affect_sorted_PNR' is your DataFram
        df_FC = affect_sorted_PNR[affect_sorted_PNR['COS_CD'] == 'FirstClass'].copy()
        df_BC = affect_sorted_PNR[affect_sorted_PNR['COS_CD'] == 'BusinessClass'].copy()
        df_PC = affect_sorted_PNR[affect_sorted_PNR['COS_CD'] == 'PremiumEconomyClass'].copy()
        df_EC = affect_sorted_PNR[affect_sorted_PNR['COS_CD'] == 'EconomyClass'].copy()
        return df_FC,df_BC,df_PC,df_EC


    matching_flights=get_available_flights_1_1(df_schedule, df_inventory, cancelled_FN, cancelled_TN, departure_date)
    affect_sorted_PNR = get_pnr_affected(df_pnr_pass, df_pnr_book, df_schedule, cancelled_FN, cancelled_TN, departure_date)
    df_FC,df_BC,df_PC,df_EC=classification(affect_sorted_PNR)

    df=complete_allocation(df_FC,df_BC,df_PC,df_EC,matching_flights)

    return df,len(affect_sorted_PNR)

















# Result Block

Just run and wait for the output CSV to be generated

In [ ]:
df,number_of_affected_PNRs=datframe_allocation(df_pnr_pass_copy, df_pnr_book_copy, df_schedule_copy,df_inventory_copy, cancelled_FN, cancelled_TN, departure_date)
df.to_csv('1-1 Reallocation_Class_Priority.csv', index=False)

# The 1-1 Allocation (Prioritizing the Flights)

We aim to prioritize reaccommodating passengers on the most essential flights, even if it means making some adjustments to the accommodation based on class.

# The 1-1 Reallocation Function Block(Prioritizing the flights of Passengers)

In [ ]:
def datframe_allocation1(df_pnr_pass, df_pnr_book, df_schedule,df_inventory, cancelled_FN, cancelled_TN, departure_date):
    def get_available_flights_1_1(schedule_df, inventory_df, cancelled_FN, cancelled_TN, departure_date):
        cancelled_schedule = df_schedule[(df_schedule['AircraftTailNumber'] == cancelled_TN) & (df_schedule['FlightNumber'] == cancelled_FN)]
        print(cancelled_schedule['ScheduleID'].iloc[0])

        cancelled_flight = df_inventory[(df_inventory['ScheduleId'] == cancelled_schedule['ScheduleID'].iloc[0]) & (df_inventory['DepartureDate'] == departure_date)]
        cancelled_flight_DepartureDateTime = cancelled_flight['DepartureDateTime'].iloc[0]
        cancelled_flight_ArrivalDateTime = cancelled_flight['ArrivalDateTime'].iloc[0]

        matching_flights = df_inventory[
        (df_inventory['DepartureAirport'] == cancelled_flight['DepartureAirport'].iloc[0]) &
        (df_inventory['ArrivalAirport'] == cancelled_flight['ArrivalAirport'].iloc[0]) &
        (df_inventory['DepartureDateTime'] > cancelled_flight_DepartureDateTime)
        ]

        columns_to_keep=['InventoryId', 'ScheduleId', 'CarrierCode', 'Dep_Key', 'FlightNumber', 'AircraftType', 'DepartureDate', 'DepartureDateTime', 'ArrivalDateTime', 'DepartureAirport', 'ArrivalAirport', 'TotalCapacity', 'TotalInventory','BookedInventory', 'Oversold', 'AvailableInventory', 'FirstClass', 'BusinessClass', 'PremiumEconomyClass', 'EconomyClass', 'FC_TotalInventory', 'FC_BookedInventory', 'FC_Oversold', 'FC_AvailableInventory', 'BC_TotalInventory', 'BC_BookedInventory', 'BC_Oversold', 'BC_AvailableInventory', 'PC_TotalInventory', 'PC_BookedInventory', 'PC_Oversold', 'PC_AvailableInventory', 'EC_TotalInventory', 'EC_BookedInventory', 'EC_Oversold', 'EC_AvailableInventory', 'FC_CD', 'BC_CD', 'PC_CD', 'EC_CD']
        matching_flights = matching_flights[columns_to_keep]

        matching_flights['Arrival_TimeDifference'] = (matching_flights['ArrivalDateTime'] - cancelled_flight_ArrivalDateTime).dt.total_seconds()/3600
        matching_flights = matching_flights.sort_values(by='Arrival_TimeDifference')
        print(matching_flights.shape)
        return matching_flights

    def process_pnr_data(df_pnr_pass, df_pnr_book, df_schedule, flight_number, tail_number, date):
        dept_time = df_schedule[(df_schedule['FlightNumber'] == flight_number) & (df_schedule['AircraftTailNumber'] == tail_number)]['DepartureTime'].iloc[0]
        date_time = date + ' ' + dept_time
        print(date_time)

        def cal_affected_persons(flt_number, date_time):
            dict={}
            affect_persons = pd.DataFrame(columns=df_pnr_book.columns)

            for i in range(len(df_pnr_book)):
                if df_pnr_book.loc[i, "RECLOC"] in dict.keys():
                    dict[df_pnr_book.loc[i, "RECLOC"]] = max(dict[df_pnr_book.loc[i, "RECLOC"]], df_pnr_book.loc[i, "SEG_SEQ"])
                else:
                    dict[df_pnr_book.loc[i, "RECLOC"]] = df_pnr_book.loc[i, "SEG_SEQ"]

                if df_pnr_book.loc[i, "FLT_NUM"] == flt_number and df_pnr_book.loc[i, "DEP_DTML"] == date_time:
                    affect_persons = affect_persons.append(df_pnr_book.loc[i])
            li = []
            for i in range(len(affect_persons)):
                if(dict[affect_persons.iloc[i]["RECLOC"]]):
                     li.append(dict[affect_persons.iloc[i]["RECLOC"]])
            affect_persons["MAX_SEG"] = li
            return affect_persons

        def calc_ssr(flight_number, date_time):
            affect_person = cal_affected_persons(flight_number, date_time)
            li = []
            for i in range(len(affect_person)):
                li.append(0)
            affect_person["count_ssr"] = li
            for i in range(len(affect_person)):
                for j in range(len(df_pnr_pass)):
                    if df_pnr_pass.loc[j, "RECLOC"] == affect_person.iloc[i]["RECLOC"]:
                        if pd.notna(df_pnr_pass["SSR_CODE_CD1"][j]) or pd.notna(df_pnr_pass["SPECIAL_NAME_CD2"][j]):
                            affect_person.iloc[i]["count_ssr"] += 1
            return affect_person

        def score_eval(affect_PNR):
            score = 0
            score += 200 * affect_PNR.loc["count_ssr"] + 50 * affect_PNR.loc["PAX_CNT"]
            score += 100 * (affect_PNR.loc["MAX_SEG"] - affect_PNR.loc["SEG_SEQ"])
            score_dict = {'A': 'J', 'D': 'J', 'J': 'J', 'B': 'F', 'F': 'F', 'S': 'Y', 'V': 'Y', 'W': 'Y', 'Z': 'Y', 'O': 'Y',
                      'S': 'Y', 'T': 'Y', 'U': 'Y', 'M': 'Y', 'N': 'Y', 'Y': 'Y'}
            if affect_PNR.loc["COS_CD"] not in score_dict.keys():
                score += 500
            elif affect_PNR.loc["COS_CD"] == 'J' or score_dict[affect_PNR.loc["COS_CD"]] == 'J':
                score += 2000
            elif affect_PNR.loc["COS_CD"] == "F" or score_dict[affect_PNR.loc["COS_CD"]] == "F":
                score += 1700
            elif affect_PNR.loc["COS_CD"] == "Y" or score_dict[affect_PNR.loc["COS_CD"]] == "Y":
                score += 1500
            elif affect_PNR.loc["COS_CD"] in ["A", "C", "K"]:
                score += 800
            return score

        def score(flight, data_time):
            affect_PNR = calc_ssr(flight, data_time)
            pnr_ranking = pd.DataFrame(columns=affect_PNR.columns.tolist() + ['score'])
            for i in range(len(affect_PNR)):
                score_val = score_eval(affect_PNR.iloc[i])
                affect_row = list(affect_PNR.iloc[i])
                affect_row.append(score_val)
                pnr_ranking = pnr_ranking.append(pd.Series(affect_row, index=pnr_ranking.columns), ignore_index=True)

            sorted_pnr_ranking = pnr_ranking.sort_values(by='score', ascending=False)
            return sorted_pnr_ranking
        return score(flight_number, date_time)

    def get_ITN_DEST(df):
        df[['SEG_SEQ', 'SEG_TOTAL']] = df[['SEG_SEQ', 'SEG_TOTAL']].apply(pd.to_numeric, errors='coerce')
        max_dest_per_recloc = df.loc[df['SEG_SEQ'] == df['SEG_TOTAL']].groupby('RECLOC')['DEST_CD'].first().reset_index()
        df = pd.merge(df, max_dest_per_recloc, on='RECLOC', how='left', suffixes=('', '_ITN_DEST'))
        return df
    def get_cluster_rank(affect_sorted_PNR):
        cluster_means = affect_sorted_PNR.groupby('DEST_CD_ITN_DEST')['score'].mean()
        rank_mapping = {cluster: rank for rank, (cluster, _) in enumerate(cluster_means.sort_values(ascending=False).iteritems(), start=1)}
        affect_sorted_PNR['CLUSTER_RANK'] = affect_sorted_PNR['DEST_CD_ITN_DEST'].map(rank_mapping)
        return affect_sorted_PNR

    def get_pnr_affected(df_pnr_pass, df_pnr_book, df_schedule, flight_number, tail_number, date):
        df_pnr_book = get_ITN_DEST(df_pnr_book)
        affect_sorted_PNR = process_pnr_data(df_pnr_pass, df_pnr_book, df_schedule, flight_number, tail_number, date)
        final_affected_PNR = get_cluster_rank(affect_sorted_PNR)
        return final_affected_PNR
    def knapsack1(left_seats, n, li, PNR_df, memo):
        if n == 0 or left_seats == 0:
            return left_seats, li

    # Check if the result is already memoized
        if (left_seats, n) in memo:
             return memo[(left_seats, n)]

        if left_seats >= PNR_df.iloc[n-1]["PAX_CNT"]:
        # Recursively compute the result and memoize it
            result1 = knapsack1(left_seats, n-1, li, PNR_df, memo)
            result2 = knapsack1(left_seats - PNR_df.iloc[n-1]["PAX_CNT"], n-1, li + [PNR_df.iloc[n-1]["RECLOC"]], PNR_df, memo)
            result = min(result1, result2, key=lambda x: x[0])
        else:
            result = knapsack1(left_seats, n-1, li, PNR_df, memo)

    # Memoize the result
        memo[(left_seats, n)] = result

        return result

    # Example usag  # Replace with your actual DataFrame

    def delete_entry(to_delete, dataframe):
        # print(dataframe)
        for i in to_delete:
            dataframe.drop(dataframe[dataframe["RECLOC"]==i].index, inplace=True, axis=0)
        return dataframe

    def partial_allocation_knapsack1(df_PNR,flights,cls):
        allocated_dict={}
        prev_cls=cls
        for i in range(len(flights)):
           seats_left,allocated_PNRs=knapsack1(flights.iloc[i][cls+"_AvailableInventory"], len(df_PNR), [], df_PNR,{})

           temp_column = cls+"_AvailableInventory"
           temp = flights.iloc[i][temp_column]
           column_index = flights.columns.get_loc(temp_column)
           flights.iat[i, column_index] = seats_left
           flights.iat[i, flights.columns.get_loc("AvailableInventory")] -= temp - seats_left
           df_PNR = delete_entry(allocated_PNRs, df_PNR)
           allocated_dict[flights.iloc[i]["InventoryId"]] = allocated_dict.get(flights.iloc[i]["InventoryId"], [])+[(pnr, cls,prev_cls) for pnr in allocated_PNRs]


           if(not df_PNR.empty):
            if(cls=='FC'):
                # prev_cls=cls
                cls='BC'
                seats_left,allocated_PNRs=knapsack1(flights.iloc[i][cls+"_AvailableInventory"], len(df_PNR), [], df_PNR,{})
                temp_column = cls+"_AvailableInventory"
                temp = flights.iloc[i][temp_column]
                column_index = flights.columns.get_loc(temp_column)
                flights.iat[i, column_index] = seats_left
                flights.iat[i, flights.columns.get_loc("AvailableInventory")] -= temp - seats_left
                df_PNR = delete_entry(allocated_PNRs, df_PNR)
                allocated_dict[flights.iloc[i]["InventoryId"]] = allocated_dict.get(flights.iloc[i]["InventoryId"], [])+[(pnr, cls,prev_cls) for pnr in allocated_PNRs]

                if(not df_PNR.empty):
                    cls='PC'
                    seats_left,allocated_PNRs=knapsack1(flights.iloc[i][cls+"_AvailableInventory"], len(df_PNR), [], df_PNR,{})
                    temp_column = cls+"_AvailableInventory"
                    temp = flights.iloc[i][temp_column]
                    column_index = flights.columns.get_loc(temp_column)
                    flights.iat[i, column_index] = seats_left
                    flights.iat[i, flights.columns.get_loc("AvailableInventory")] -= temp - seats_left
                    df_PNR = delete_entry(allocated_PNRs, df_PNR)
                    allocated_dict[flights.iloc[i]["InventoryId"]] = allocated_dict.get(flights.iloc[i]["InventoryId"], [])+[(pnr, cls,prev_cls) for pnr in allocated_PNRs]

                if(not df_PNR.empty):
                    cls='EC'
                    seats_left,allocated_PNRs=knapsack1(flights.iloc[i][cls+"_AvailableInventory"], len(df_PNR), [], df_PNR,{})
                    temp_column = cls+"_AvailableInventory"
                    temp = flights.iloc[i][temp_column]
                    column_index = flights.columns.get_loc(temp_column)
                    flights.iat[i, column_index] = seats_left
                    flights.iat[i, flights.columns.get_loc("AvailableInventory")] -= temp - seats_left
                    df_PNR = delete_entry(allocated_PNRs, df_PNR)
                    allocated_dict[flights.iloc[i]["InventoryId"]] = allocated_dict.get(flights.iloc[i]["InventoryId"], [])+[(pnr, cls,prev_cls) for pnr in allocated_PNRs]

            elif(cls=='BC'):
                # prev_cls=cls
                cls='PC'
                seats_left,allocated_PNRs=knapsack1(flights.iloc[i][cls+"_AvailableInventory"], len(df_PNR), [], df_PNR,{})
                temp_column = cls+"_AvailableInventory"
                temp = flights.iloc[i][temp_column]
                column_index = flights.columns.get_loc(temp_column)
                flights.iat[i, column_index] = seats_left
                flights.iat[i, flights.columns.get_loc("AvailableInventory")] -= temp - seats_left
                df_PNR = delete_entry(allocated_PNRs, df_PNR)
                allocated_dict[flights.iloc[i]["InventoryId"]] = allocated_dict.get(flights.iloc[i]["InventoryId"], [])+[(pnr, cls,prev_cls) for pnr in allocated_PNRs]
                if(not df_PNR.empty):
                    cls='EC'
                    seats_left,allocated_PNRs=knapsack1(flights.iloc[i][cls+"_AvailableInventory"], len(df_PNR), [], df_PNR,{})
                    temp_column = cls+"_AvailableInventory"
                    temp = flights.iloc[i][temp_column]
                    column_index = flights.columns.get_loc(temp_column)
                    flights.iat[i, column_index] = seats_left
                    flights.iat[i, flights.columns.get_loc("AvailableInventory")] -= temp - seats_left
                    df_PNR = delete_entry(allocated_PNRs, df_PNR)
                    allocated_dict[flights.iloc[i]["InventoryId"]] = allocated_dict.get(flights.iloc[i]["InventoryId"], [])+[(pnr, cls,prev_cls) for pnr in allocated_PNRs]

                if(not df_PNR.empty):
                    cls='FC'
                    seats_left,allocated_PNRs=knapsack1(flights.iloc[i][cls+"_AvailableInventory"], len(df_PNR), [], df_PNR,{})
                    temp_column = cls+"_AvailableInventory"
                    temp = flights.iloc[i][temp_column]
                    column_index = flights.columns.get_loc(temp_column)
                    flights.iat[i, column_index] = seats_left
                    flights.iat[i, flights.columns.get_loc("AvailableInventory")] -= temp - seats_left
                    df_PNR = delete_entry(allocated_PNRs, df_PNR)
                    allocated_dict[flights.iloc[i]["InventoryId"]] = allocated_dict.get(flights.iloc[i]["InventoryId"], [])+[(pnr, cls,prev_cls) for pnr in allocated_PNRs]

            elif(cls=='PC'):
                # prev_cls=cls
                cls='EC'
                seats_left,allocated_PNRs=knapsack1(flights.iloc[i][cls+"_AvailableInventory"], len(df_PNR), [], df_PNR,{})
                temp_column = cls+"_AvailableInventory"
                temp = flights.iloc[i][temp_column]
                column_index = flights.columns.get_loc(temp_column)
                flights.iat[i, column_index] = seats_left
                flights.iat[i, flights.columns.get_loc("AvailableInventory")] -= temp - seats_left
                df_PNR = delete_entry(allocated_PNRs, df_PNR)
                allocated_dict[flights.iloc[i]["InventoryId"]] = allocated_dict.get(flights.iloc[i]["InventoryId"], [])+[(pnr, cls,prev_cls) for pnr in allocated_PNRs]

                if(not df_PNR.empty):
                    cls='BC'
                    seats_left,allocated_PNRs=knapsack1(flights.iloc[i][cls+"_AvailableInventory"], len(df_PNR), [], df_PNR,{})
                    temp_column = cls+"_AvailableInventory"
                    temp = flights.iloc[i][temp_column]
                    column_index = flights.columns.get_loc(temp_column)
                    flights.iat[i, column_index] = seats_left
                    flights.iat[i, flights.columns.get_loc("AvailableInventory")] -= temp - seats_left
                    df_PNR = delete_entry(allocated_PNRs, df_PNR)
                    allocated_dict[flights.iloc[i]["InventoryId"]] = allocated_dict.get(flights.iloc[i]["InventoryId"], [])+[(pnr, cls,prev_cls) for pnr in allocated_PNRs]

                if(not df_PNR.empty):
                    cls='FC'
                    seats_left,allocated_PNRs=knapsack1(flights.iloc[i][cls+"_AvailableInventory"], len(df_PNR), [], df_PNR,{})
                    temp_column = cls+"_AvailableInventory"
                    temp = flights.iloc[i][temp_column]
                    column_index = flights.columns.get_loc(temp_column)
                    flights.iat[i, column_index] = seats_left
                    flights.iat[i, flights.columns.get_loc("AvailableInventory")] -= temp - seats_left
                    df_PNR = delete_entry(allocated_PNRs, df_PNR)
                    allocated_dict[flights.iloc[i]["InventoryId"]] = allocated_dict.get(flights.iloc[i]["InventoryId"], [])+[(pnr, cls,prev_cls) for pnr in allocated_PNRs]

            elif(cls=='EC' ):
                # prev_cls=cls
                cls='PC'
                seats_left,allocated_PNRs=knapsack1(flights.iloc[i][cls+"_AvailableInventory"], len(df_PNR), [], df_PNR,{})
                temp_column = cls+"_AvailableInventory"
                temp = flights.iloc[i][temp_column]
                column_index = flights.columns.get_loc(temp_column)
                flights.iat[i, column_index] = seats_left
                flights.iat[i, flights.columns.get_loc("AvailableInventory")] -= temp - seats_left
                df_PNR = delete_entry(allocated_PNRs, df_PNR)
                allocated_dict[flights.iloc[i]["InventoryId"]] = allocated_dict.get(flights.iloc[i]["InventoryId"], [])+[(pnr, cls,prev_cls) for pnr in allocated_PNRs]

                if(not df_PNR.empty):
                    cls='BC'
                    seats_left,allocated_PNRs=knapsack1(flights.iloc[i][cls+"_AvailableInventory"], len(df_PNR), [], df_PNR,{})
                    temp_column = cls+"_AvailableInventory"
                    temp = flights.iloc[i][temp_column]
                    column_index = flights.columns.get_loc(temp_column)
                    flights.iat[i, column_index] = seats_left
                    flights.iat[i, flights.columns.get_loc("AvailableInventory")] -= temp - seats_left
                    df_PNR = delete_entry(allocated_PNRs, df_PNR)
                    allocated_dict[flights.iloc[i]["InventoryId"]] = allocated_dict.get(flights.iloc[i]["InventoryId"], [])+[(pnr, cls,prev_cls) for pnr in allocated_PNRs]

                if(not df_PNR.empty):
                    cls='FC'
                    seats_left,allocated_PNRs=knapsack1(flights.iloc[i][cls+"_AvailableInventory"], len(df_PNR), [], df_PNR,{})
                    temp_column = cls+"_AvailableInventory"
                    temp = flights.iloc[i][temp_column]
                    column_index = flights.columns.get_loc(temp_column)
                    flights.iat[i, column_index] = seats_left
                    flights.iat[i, flights.columns.get_loc("AvailableInventory")] -= temp - seats_left
                    df_PNR = delete_entry(allocated_PNRs, df_PNR)
                    allocated_dict[flights.iloc[i]["InventoryId"]] = allocated_dict.get(flights.iloc[i]["InventoryId"], [])+[(pnr, cls,prev_cls) for pnr in allocated_PNRs]

        return allocated_dict








    def merge_dict(dict1,dict2):
        list1=dict1.keys()
        for i in list1:
            dict1[i]=dict1[i]+dict2[i]
        return dict1

    def convert_dict_to_dataframe(allocated_dict):
        data = []
        for inventory_id, allocations in allocated_dict.items():
            for allocation in allocations:
                pnr, class_name,prev_clssname = allocation
                data.append({'PNR': pnr, 'InventoryId': inventory_id, 'New_ClassName': class_name,'Prev_ClassName': prev_clssname})

        df = pd.DataFrame(data)
        return df

    def the_alloactions(df_FC_copy,df_BC_copy,df_PE_copy,df_EC_copy,flights):
        partial_FC_alloaction=partial_allocation_knapsack1(df_FC_copy, flights, "FC")
        partial_BC_alloaction=partial_allocation_knapsack1(df_BC_copy, flights, "BC")
        partial_PE_alloaction=partial_allocation_knapsack1(df_PE_copy, flights, "PC")
        partial_EC_alloaction=partial_allocation_knapsack1(df_EC_copy, flights, "EC")
        PNR_NOT_Allocated=df_FC_copy['RECLOC'].tolist()
        PNR_NOT_Allocated=PNR_NOT_Allocated+df_BC_copy['RECLOC'].tolist()+df_PE_copy['RECLOC'].tolist()+df_EC_copy['RECLOC'].tolist()

        df=convert_dict_to_dataframe(partial_FC_alloaction)
        df=pd.concat([df, convert_dict_to_dataframe(partial_BC_alloaction)], ignore_index=True)
        df=pd.concat([df, convert_dict_to_dataframe(partial_PE_alloaction)], ignore_index=True)
        df=pd.concat([df, convert_dict_to_dataframe(partial_EC_alloaction)], ignore_index=True)
        new_df=pd.DataFrame({'PNR':PNR_NOT_Allocated})

        print(df)

        print(flights)

        merged_df = pd.merge(df, flights, on='InventoryId', how='inner')

        result_df= new_df.set_index('PNR').combine_first(merged_df.set_index('PNR')).reset_index()


        return result_df


    def classification(affect_sorted_PNR):
        # Assuming 'affect_sorted_PNR' is your DataFram
        df_FC = affect_sorted_PNR[affect_sorted_PNR['COS_CD'] == 'FirstClass'].copy()
        df_BC = affect_sorted_PNR[affect_sorted_PNR['COS_CD'] == 'BusinessClass'].copy()
        df_PC = affect_sorted_PNR[affect_sorted_PNR['COS_CD'] == 'PremiumEconomyClass'].copy()
        df_EC = affect_sorted_PNR[affect_sorted_PNR['COS_CD'] == 'EconomyClass'].copy()
        return df_FC,df_BC,df_PC,df_EC


    matching_flights=get_available_flights_1_1(df_schedule, df_inventory, cancelled_FN, cancelled_TN, departure_date)
    affect_sorted_PNR = get_pnr_affected(df_pnr_pass, df_pnr_book, df_schedule, cancelled_FN, cancelled_TN, departure_date)
    df_FC,df_BC,df_PC,df_EC=classification(affect_sorted_PNR)

    df=the_alloactions(df_FC,df_BC,df_PC,df_EC,matching_flights)

    return df,len(affect_sorted_PNR)

















Just run and wait for the output CSV to be generated

In [ ]:
df1,number_of_affected_PNRs=datframe_allocation1(df_pnr_pass_copy1, df_pnr_book_copy1, df_schedule_copy1,df_inventory_copy1, cancelled_FN, cancelled_TN, departure_date)
df1.to_csv('1-1 Reallocation_Flight_Priority.csv', index=False)

SCH-ZZ-1589980
(49, 41)
04/21/2024 16:42


<ipython-input-11-7a3e7a58bdc8>:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  affect_persons = affect_persons.append(df_pnr_book.loc[i])
<ipython-input-11-7a3e7a58bdc8>:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  affect_persons = affect_persons.append(df_pnr_book.loc[i])
<ipython-input-11-7a3e7a58bdc8>:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  affect_persons = affect_persons.append(df_pnr_book.loc[i])
<ipython-input-11-7a3e7a58bdc8>:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  affect_persons = affect_persons.append(df_pnr_book.loc[i])
<ipython-input-11-7a3e7a58bdc8>:40: FutureWarning: The frame.append method is de

       PNR     InventoryId New_ClassName Prev_ClassName
0   EXXS33  INV-ZZ-1957001            FC             FC
1   ELBU85  INV-ZZ-1957001            BC             FC
2   NHIY83  INV-ZZ-1957001            BC             FC
3   HYTZ86  INV-ZZ-1957001            BC             FC
4   CWTV97  INV-ZZ-1957001            BC             FC
5   RHLA51  INV-ZZ-1957001            PC             BC
6   HJCQ26  INV-ZZ-1957001            PC             BC
7   LFJE82  INV-ZZ-1957001            PC             BC
8   QPXJ84  INV-ZZ-1957001            EC             BC
9   ZGRB68  INV-ZZ-1957001            EC             BC
10  SXGH40  INV-ZZ-1957001            EC             BC
11  IWRM28  INV-ZZ-1957001            EC             BC
12  XLES89  INV-ZZ-1957001            EC             BC
13  MPCS79  INV-ZZ-1957001            EC             BC
14  TIKK22  INV-ZZ-1957001            EC             PC
15  WXXO13  INV-ZZ-1957001            EC             PC
16  VJIH63  INV-ZZ-4434695            FC        

In [ ]:
number_of_affected_PNRs

32